In [3]:
%load_ext autoreload
%autoreload 2

from warnings import filterwarnings

import torch
import torch.nn as nn

import config_iProL_test as config
from dataset import RegulonDataset
from dataset import get_excel, get_complete_data, get_split_dataset
from dataset import get_datasets, get_cv_dataloader_list, feature_exaction
from feature_extraction import Kmer
from module import Longformer_base_Net, Longformer_base_lstm_Net, Bert_base_lstm_Net
from tools import get_scoring_result, drawLoss, check_result_dir, get_config_info, save_score
from tools_fit import model_fit

/home/pbc/Pycharm Projects/myNewPro/src/feature_extraction/__init__.py


In [4]:
sheet = get_excel(version='', sheet_name='dataset')
complete_data = get_complete_data(sheet)
dataset_train, _, _ = get_datasets(complete_data)

sheet = get_excel(version='11.1', sheet_name='test_0.85')
complete_data = get_complete_data(sheet)
dataset_test, _, _ = get_datasets(complete_data)

In [5]:
train_loader, test_loader = feature_exaction(config, [dataset_train, dataset_test], "Pro")

PBC[test] --- kmer = 2
train len: 6764 test len: 395


In [6]:
filterwarnings("ignore")
check_result_dir(config, "test")

model = Longformer_base_lstm_Net(config.model_name, config.device,
                                 config.add_special_tokens,
                                 bidirectional=config.bidirectional,
                                 num_layers=config.num_layers)
model.to(config.device)

# 这里是一般情况，共享层往往不止一层，所以做一个for循环
if hasattr(model, 'longformer') and config.not_fine_tuning:
    for para in model.longformer.parameters():
        para.requires_grad = False
    # print(module.parameters())

optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=config.lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 50, gamma=0.6)
criterion = nn.BCELoss()

model_fit(config, model, train_loader, test_loader, optimizer, criterion, scheduler, is_test=True)

Ex result dir /home/pbc/Pycharm Projects/myNewPro/src/../result/longformer/_train[kmer2_bs32_lr0.0005_epoch250]_pm[astFalse_nftTrue_levelbase]_lstm[biTrue_nl1]/test_0602225310_Pro_test/test has been created!


Some weights of the model checkpoint at /home/pbc/Pycharm Projects/myNewPro/pre-model/longformer/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


>>>>>>>>>>『START』[lr:[0.0005]]>>>>>>>>>>>>
『Computer device』nvidia:test
『dataset』kmer: 2, cv: test
『train config』epochs: 250, lr: 0.0005, batch_size: 32, device: cuda
『pre-model config [True]』add_special_tokens: False, not_fine_tuning: True, model_name: longformer-base-4096
『lstm config [True]』bidirectional: True, num_layers: 1
『ex result config』ex_result_dir: /home/pbc/Pycharm Projects/myNewPro/src/../result/longformer/_train[kmer2_bs32_lr0.0005_epoch250]_pm[astFalse_nftTrue_levelbase]_lstm[biTrue_nl1]/test_0602225310_Pro_test
epoch 1 [212/212] total_loss = 0.5515365675091743 lr = 0.0005, total=6764, TP=2573, TN=2317, FP=1065, FN=809; precision=0.707, recall=0.761, Sn=0.761, Sp=0.685
accuracy: (test=0.723) matthews_corrcoef: (test=0.447) f1: (test=0.733) 
epoch 1 [13/13] total_loss = 0.43040327842418963 total=395, TP=316, TN=0, FP=0, FN=79; precision=1.000, recall=0.800, Sn=0.800, Sp=nan
accuracy: (test=0.800) matthews_corrcoef: (test=0.000) f1: (test=0.889) 

epoch 2 [212/212] total_